Welcome to the example docs for diff_classifier.  This example will walk you through using diff_classifier to track particle trajectories from a video collected via fluorescence microscopy, or some other imaging technique.  

# Loading and checking video

In order to properly track the video, the user must enter the following relevant information regarding the video to be tracked:
* **prefix**: the name of the file to be tracked, minus the file extension.  Typically tif files are the best to track.
* **local_folder**: the location of the video to be tracked.
* **ires**: the desired resolution of sub-videos to be tracked.  Since the images to be tracked may be too large to analyze all at once, the user may need to crop images to perform correctly.  512 x 512 pixels is the standard size.
* **frames**: number of frames contained in the video.  

In [1]:
import os.path as op
from diff_classifier import aws as aws

In [2]:
prefix = 'P1_s2_XY2_v'
local_folder = '../diff_classifier/data/Nano_DDS_Figures'
rfolder = '08_07_18_P35_PNN_tracking'
ires = 512
frames = 651
aws.download_s3('{}/{}.tif'.format(rfolder, prefix), '{}/{}.tif'.format(local_folder, prefix), bucket_name='hpontes.data')
#filename = '{}.tif'.format(prefix)
#local_name = op.join(local_folder, filename)

In [3]:
filename = '{}.tif'.format(prefix)
local_name = op.join(local_folder, filename)

In [4]:
import skimage.io as sio
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
example = sio.imread(local_name)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
ax.imshow(example[649, :, :], cmap='gray')
ax.axis('off')

(-0.5, 2047.5, 2043.5, -0.5)

# Splitting images prior to analysis

There is an upper limit on image resolution depending on the available RAM on your machine.  I typically limit analyses to images less than 1024 x 1024 pixels, and usually use 512 x 512 pixel images to be consistent.  The large images from the microscope in our lab are 2048 x 2044, resulting in 16 512 x 512 images, with a black band of 0s of the bottom 4 images.

In [ ]:
import diff_classifier.imagej as ij

names = ij.partition_im(local_name)

In [ ]:
print(names)

In [ ]:
names = []
for row in range(4):
    for col in range(4):
        names.append('{}_{}_{}'.format(prefix, row, col))

In [ ]:
print(names)

In [ ]:
for name in names:
    aws.upload_s3('{}/{}.tif'.format(local_folder, name), '{}/{}.tif'.format(rfolder, name), bucket_name='hpontes.data')
    print("Done with splitting.  Should output file of name {}".format(local_folder+'/'+name))

In [ ]:
names = []
for i in range(0, 4):
    for j in range(0, 4):
        names.append('{}_{}_{}.tif'.format(prefix, i, j))

# Particle tracking

Particle tracking is performed with a Trackmate script as exemplified [here](https://imagej.net/Scripting_TrackMate).  The function track allows the user to select relevant tracking parameters.  Current iteration of the code limits tracking to the DoG detector and the Sparse LAP tracker.  The following tracking parameters must be specified for tracking:
* **template**: location of template file containing Trackmate script. Default Trackmate script is used if set to None.
* **fiji_bin**: location of Fiji bin.  Fiji is required for tracking.

* **radius**: estimated particle radius in pixels. Can take half values.
* **threshold**: detection cutoff.  Default is 0.
* **do_median_filtering**: if set to True, will perform a median filter before tracking.
* **quality**: quality cutoff level.
* **x**: Upper x coordinate threshold.  If used, will exclude spots that have x coordinate greater than specified value.
* **y**: Upper y coordinate threshold.
* **ylo**: Lower y coordinate threshold.
* **median_intensity**: currently not used.
* **snr**: Signal-to-noise ratio cutoff.  Default is 0.
* **linking_max_distance**: max distance in pixels a particle can travel in a single frame.
* **gap_closing_max_distance**: max distance in pixels a particle can travel and be picked up by detector when a frame is skipped.
* **max_frame_gap**: number of frames a particle can disappear and still be linked.
* **track_displacement**: lower duration cutoff in frames. Tracks must be this many frames long to be included in final dataset.

Quality is the trickiest parameter to select, and can make or break a successful tracking iteration.  The key benefit to this software package is the ability to track large numbers of files at once.  If your files are similar enough, you can get away with using a single quality value for all videos.  In this case, I used two quality values depending on the mean intensity of the input image, which I found correlated with the best quality values for the tracking settings. In another notebook, I use a training dataset to predict quality based on the input mean intensity of the image to be tracked.

In [ ]:
print(names)

In [ ]:
print(names[0].split('.')[0])

In [ ]:
print(names[0].split('_'))
print(names[0].split('.')[0].split('_')[4])
print(names[0].split('.')[0].split('_')[5])

In [ ]:
import diff_classifier.imagej as ij

In [ ]:
ij.track()

In [ ]:
print(local_folder)

In [ ]:
for name in names:
    outfile = 'Traj_' + name.split('.')[0] + '.csv'
    local_im = op.join(local_folder, name)
    
    row = int(name.split('.')[0].split('_')[4])
    col = int(name.split('.')[0].split('_')[5])

#     try:
#         aws.download_s3(remote_folder+'/'+outfile, outfile)
#     except:
#    test_intensity = ij.mean_intensity(local_im)
#    if test_intensity > 500:
#        quality = 245
#    else:
#        quality = 4.5

    quality = 3.38
    
    if row==3:
        y = 485
    else:
        y = 511

    ij.track(local_im, outfile, template=None, fiji_bin=None, radius=5.0, threshold=0.0, 
          do_median_filtering=False, quality=quality, xdims=(0, 511), ydims=(1, y), snr=0.0, 
          linking_max_distance=10.0, gap_closing_max_distance=12.0, max_frame_gap=2, track_displacement=11.8)

    print("Done with tracking.  Should output file of name {}".format(local_folder+'/'+outfile))

In [ ]:
import os
print(os.listdir('../diff_classifier/data/Nano_DDS_Figures'))

In [ ]:
# uploading trajectory csv files up to s3
for name in names:
    outfile = 'Traj_' + name.split('.')[0] + '.csv'
    aws.upload_s3(outfile, op.join(rfolder, outfile), bucket_name='hpontes.data')
    print("Done with uploading.  Should output file of name {}".format(local_folder+'/'+outfile))

# Calculating MSDs and features

This package uses pandas dataframes to perform MSD and feature calculations.  The first dataframe contains the xy coordinates and the MSDs of each trajectory at each frame.  The second dataframe contains a single row for each trajectory and associated features of each trajectory.

This bit of code also only creates a single dataframe for each original input video, rather than creating one for each sub-video after splitting up the images.  They're a lot easier to keep track of that way and minimized total files created.

In [ ]:
print(prefix)
print(names)
print(outfile)
print(outfile.split('.')[0].split('_'))
print(outfile.split('.')[0].split('_')[5])
print(outfile.split('.')[0].split('_')[6])

In [ ]:
import diff_classifier.utils as ut
import diff_classifier.msd as msd
import diff_classifier.features as ft

msd_file = 'msd_{}.csv'.format(prefix)
ft_file = 'features_{}.csv'.format(prefix)

counter = 0
for name in names:
    outfile = 'Traj_' + name.split('.')[0] + '.csv'
    row = int(outfile.split('.')[0].split('_')[5])
    col = int(outfile.split('.')[0].split('_')[6])

    filename = "Traj_{}_{}_{}.csv".format(prefix, row, col)
    local_name = filename

    if counter == 0:
        to_add = ut.csv_to_pd(local_name)
        to_add['X'] = to_add['X'] + ires*col
        to_add['Y'] = ires - to_add['Y'] + ires*(3-row)
        merged = msd.all_msds2(to_add, frames=frames)
    else: 

        if merged.shape[0] > 0:
            to_add = ut.csv_to_pd(local_name)
            to_add['X'] = to_add['X'] + ires*col
            to_add['Y'] = ires - to_add['Y'] + ires*(3-row)
            to_add['Track_ID'] = to_add['Track_ID'] + max(merged['Track_ID']) + 1
        else:
            to_add = ut.csv_to_pd(local_name)
            to_add['X'] = to_add['X'] + ires*col
            to_add['Y'] = ires - to_add['Y'] + ires*(3-row)
            to_add['Track_ID'] = to_add['Track_ID']

        merged = merged.append(msd.all_msds2(to_add, frames=frames))
        print('Done calculating MSDs for row {} and col {}'.format(row, col))
    counter = counter + 1

merged.to_csv(msd_file)
#aws.upload_s3(msd_file, op.join(rfolder, msd_file), bucket_name='mckenna.data')
merged_ft = ft.calculate_features(merged)
merged_ft.to_csv(ft_file)
#aws.upload_s3(ft_file, op.join(rfolder, ft_file), bucket_name='mckenna.data')

In [ ]:
aws.upload_s3(msd_file, op.join(rfolder, msd_file), bucket_name='hpontes.data')
aws.upload_s3(ft_file, op.join(rfolder, ft_file), bucket_name='hpontes.data')

In [ ]:
print(prefix)
print(msd_file)

In [ ]:
print(ft_file)

In [ ]:
merged.head()

In [ ]:
merged_ft.head()

In [ ]:
print(rfolder)

# Trajectory visualization

Diff_classifier includes several visualization tools for trajectories and features.  First is a map of the trajectories as they appear in the video

In [ ]:
import diff_classifier.heatmaps as hm


hm.plot_trajectories(prefix, remote_folder=rfolder, bucket='hpontes.data')

In [ ]:
import diff_classifier.heatmaps_Nano_DDS as hmdd

hmdd.plot_trajectories(prefix, remote_folder=rfolder, upload = True, bucket='hpontes.data')

In [ ]:
import pandas as pd

merged = pd.read_csv('msd_{}.csv'.format(prefix))
particles = int(max(merged['Track_ID']))
ires = 512

fig = plt.figure(figsize=(12, 12))
for part in range(0, particles):
    x = merged[merged['Track_ID'] == part]['X']
    y = merged[merged['Track_ID'] == part]['Y']
    plt.plot(x, y, linewidth=2)

plt.xlim(ires*1, ires*4)
plt.ylim(ires*2, ires*4)
plt.axis('off')

Next, the user can plot heatmaps of the trajectories with colormaps scaled to a particular feature.  In this case, the heatmap is scaled to the boundedness of the trajectories.

In [ ]:
feature = 'boundedness'
hm.plot_heatmap(prefix, feature=feature, vmin=merged_ft[feature].quantile(0.1), vmax=merged_ft[feature].quantile(0.9), remote_folder=rfolder, bucket='mckenna.data')

This information can also be displayed in the form of a scatterplot:

In [ ]:
hm.plot_scatterplot(prefix, feature=feature, vmin=merged_ft[feature].quantile(0.1), vmax=merged_ft[feature].quantile(0.9))

The user can also plot a distribution of the diffusion coefficients:

In [ ]:
hm.plot_histogram(prefix, remote_folder=rfolder, bucket='mckenna.data')

One useful check for data quality is the number of particles per frame.  Inevitably, this is a decreating function with respect to time lag.  However, if the linking step was performed successfully, the curve shouldn't be too steep.

In [ ]:
hm.plot_particles_in_frame(prefix, y_range=10000, remote_folder=rfolder, bucket='mckenna.data')

Finally, the user can plot the MSDs of the individual trajectories.  This function also calculates the geometrically averaged MSDs and the standard deviation of the logs of the MSDs.

In [ ]:
gmean1, gSEM1 = hm.plot_individual_msds(prefix, x_range= 2, umppx=0.07, alpha=0.05, remote_folder=rfolder, upload=True, bucket='hpontes.data')

In [ ]:
print(rfolder)
print(prefix)
import os
print(os.getcwd())
print(folder)

In [ ]:
aws.upload_s3('./geomean_{}.csv'.format(prefix), rfolder+'/'+'geomean_'+prefix+'.csv', bucket_name='hpontes.data')
aws.upload_s3('./geoSEM_{}.csv'.format(prefix), rfolder+'/'+'geoSEM_'+prefix+'.csv', bucket_name='hpontes.data')
aws.upload_s3('./msds_{}.png'.format(prefix), rfolder+'/'+'msds_'+prefix+'.png', bucket_name='hpontes.data')